In [1]:
# @title Análise de Breast Cancer e Classificação
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

# 1. Carregamento dos dados
# Nota: Certifique-se de fazer o upload do arquivo 'breast-cancer.csv' no Colab
try:
    df = pd.read_csv("breast-cancer.csv")
except FileNotFoundError:
    print("Erro: O arquivo 'breast-cancer.csv' não foi encontrado no diretório local.")

    # Criando um dataset dummy caso o usuário esqueça o upload para o código não quebrar totalmente
    # (Este bloco 'else' seria desnecessário se o arquivo estiver lá)

# O dataset usa '?' para dados faltantes. Vamos substituir por NaN do numpy
df.replace("?", np.nan, inplace=True)

print("=== 1. Atributos com dados ausentes ===")
missing_info = df.isnull().sum()
missing_cols = missing_info[missing_info > 0]
for col_name, count in missing_cols.items():
    print(f"Atributo: {col_name} | Quantidade ausente: {count}")

print("\n=== 2. Frequência das Classes ===")
class_counts = df["Class"].value_counts()
print(class_counts)

print("\n=== 3 & 4. Faixa etária mais frequente por classe ===")
ct = pd.crosstab(df["age"], df["Class"])
print(ct)
print(f"Mais frequente para recurrence-events: {ct['recurrence-events'].idxmax()}")
print(
    f"Mais frequente para no-recurrence-events: {ct['no-recurrence-events'].idxmax()}"
)

print("\n=== 5. Moda dos atributos com valores ausentes ===")
modes = df[missing_cols.index].mode().iloc[0]
print(modes)

# === 6. Tarefa de Classificação ===

# A) Substituir valores ausentes pela moda
df_clean = df.copy()
for col in missing_cols.index:
    df_clean[col] = df_clean[col].fillna(modes[col])

# Separar X e y
X = df_clean.drop("Class", axis=1)
y = df_clean["Class"]

# B) Converter categóricos usando OneHotEncoder
# BinaryIOencoder mencionado no prompt parece ser OneHotEncoder com saída binária
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_encoded = encoder.fit_transform(X)

print("\n=== Resultados da Classificação (10-Fold CV) ===")

# Algoritmos
# J48 é a implementação Java do C4.5. No Scikit-Learn, usamos DecisionTreeClassifier (CART)
# Utilizamos criterion='entropy' para ficar mais próximo da lógica do J48 (ganho de informação)
clf_j48 = DecisionTreeClassifier(criterion="entropy", random_state=42)
clf_nb = GaussianNB()

# Cross Validation - 10 Folds
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Execução e Avaliação
acc_j48 = cross_val_score(clf_j48, X_encoded, y, cv=kfold, scoring="accuracy")
acc_nb = cross_val_score(clf_nb, X_encoded, y, cv=kfold, scoring="accuracy")

print(f"Acurácia Média Árvore de Decisão (J48): {acc_j48.mean() * 100:.2f}%")
print(f"Acurácia Média Naive Bayes: {acc_nb.mean() * 100:.2f}%")

if acc_j48.mean() > acc_nb.mean():
    print("\nMelhor algoritmo: Árvore de Decisão")
else:
    print("\nMelhor algoritmo: Naive Bayes")

print("\n=== 7. Critérios de Ligação (Linkage) ===")
print("1. Single Linkage (Vizinho mais próximo)")
print("2. Complete Linkage (Vizinho mais distante)")
print("3. Average Linkage (Média das distâncias)")

=== 1. Atributos com dados ausentes ===
Atributo: node-caps | Quantidade ausente: 8
Atributo: breast-quad | Quantidade ausente: 1

=== 2. Frequência das Classes ===
Class
no-recurrence-events    201
recurrence-events        85
Name: count, dtype: int64

=== 3 & 4. Faixa etária mais frequente por classe ===
Class  no-recurrence-events  recurrence-events
age                                           
20-29                     1                  0
30-39                    21                 15
40-49                    63                 27
50-59                    71                 25
60-69                    40                 17
70-79                     5                  1
Mais frequente para recurrence-events: 40-49
Mais frequente para no-recurrence-events: 50-59

=== 5. Moda dos atributos com valores ausentes ===
node-caps            no
breast-quad    left_low
Name: 0, dtype: object

=== Resultados da Classificação (10-Fold CV) ===
Acurácia Média Árvore de Decisão (J48): 63.92%
Acu